<a href="https://colab.research.google.com/github/JayMalde/Deep-Learning-QA-Project/blob/main/QnA_based_on_Finacial_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/DL Project

/content/gdrive/MyDrive/DL Project


In [ ]:
data = pd.read_csv("FiQA_train_doc_final.tsv",delimiter="\t")
data.head()

,Unnamed: 0,docid,doc,timestamp
0,0,3,I'm not saying I don't like the idea of on-the...,Oct 03 '12 at 14:56
1,1,31,So nothing preventing false ratings besides ad...,Sep 01 '17 at 13:36
2,2,56,You can never use a health FSA for individual ...,Jun 9 '14 at 17:37
3,3,59,Samsung created the LCD and other flat screen ...,Dec 27 at 01:37
4,4,63,Here are the SEC requirements: The federal sec...,Jul 14 '14 at 8:10


In [ ]:
data.shape

(57638, 4)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import gensim
from gensim.models import Word2Vec
import numpy as np
import nltk
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy
from scipy import spatial
from nltk.tokenize.toktok import ToktokTokenizer
import re

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

In [ ]:
sentences=data['doc']
sentences.head()

0    I'm not saying I don't like the idea of on-the...
1    So nothing preventing false ratings besides ad...
2    You can never use a health FSA for individual ...
3    Samsung created the LCD and other flat screen ...
4    Here are the SEC requirements: The federal sec...
Name: doc, dtype: object

In [ ]:
df=data.groupby('doc').filter(lambda x:len(x)<500).reset_index(drop=True)


In [ ]:
f=df['doc'].unique()

In [ ]:
# import gensim package
import gensim

# load the saved model
model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
# Step 7.1.4 : Create IR system

# Now we build the information retrieval system:

# Preprocessing
def remove_stopwords(text, is_lower_case=False):
 pattern = r'[^a-zA-Z0-9\s]'
 text = re.sub(pattern, "" , text)
 

 tokens = tokenizer.tokenize(text)
 tokens = [token.strip() for token in tokens]
 if is_lower_case:
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  
 else:
  filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  
 return filtered_text


# Function to get the embedding vector for n dimension, we have used "300"
def get_embedding(word):
 if word in model.wv.vocab:
  return model[word]
 else:
  return np.zeros(300)

In [ ]:

print(len(tokenizer.tokenize(f)))


519


In [ ]:
out_dict = {}
for sen in f:  # this loop will pick one sentence at a time from fin (final document)
 average_vector = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(remove_stopwords(sen))]), axis=0))
 dict = { sen : (average_vector) }
 out_dict.update(dict)

# Function to calculate the similarity between the query vector and document vector
def get_sim(query_embedding, average_vector_doc):
 sim = [(1 - scipy.spatial.distance.cosine(query_embedding, average_vector_doc))]
 return sim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
print(out_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Rank all the documents based on the similarity to get relevant docs
def Ranked_documents(query):
 query_words = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(query.lower())],dtype=float), axis=0))
 rank = []
 
 for k,v in out_dict.items():
  rank.append((k, get_sim(query_words, v)))
 
 rank = sorted(rank,key=lambda t: t[1], reverse=True)
 print('Ranked Documents :')
 return rank[:4]

In [ ]:
Ranked_documents("What should I look at before investing in a start-up?")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Ranked Documents :


[("First off, you have done very well to be in your financial position at your age. Congratulations.  I first started investing seriously about 10 years ago, and when I started, I had a similar attitude to you. Learning how to invest is a journey, and it will take you a while to learn both the intellectual and emotional sides of investing. First off, there is nothing wrong with having a chunk of cash that you aren't investing effectively. It is far better to be losing earning power WRT inflation that it is to make a bad investment, where you can lose all your money quite quickly. I have perhaps 15% of my capital just sitting around right now because I don't have any place where I'm excited to put it. For your IRA, I would look at the options you have, and choose one that is reasonably well diversified and has low costs. In most cases, an index fund is a reasonable choice. My 401K goes into an S&P 500 index fund, and I don't have to worry about it. Beyond that, I suggest spending some t

In [ ]:
Ranked_documents("Are Stcoks good option to invest your money?")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Ranked Documents :


[('You have a few correlated questions here:  Yes you can. There are only a few investment strategies that require a minimum contribution and those aren\'t ones that would get a blanket recommendation anyway. Investing in bonds or stocks is perfectly possible with limited funds. You\'re never too young to start. The power of interest means that the more time you give your money to grow, the larger your eventual gains will be (provided your investment is beating inflation). If your financial situation allows it, it makes sense to invest money you don\'t need immediately, which brings us to: This is the one you have to look at most. You\'re young but have a nice chunk of cash in a savings account. That money won\'t grow much and you could be losing purchasing power to inflation but on the other hand that money also isn\'t at risk. While there are dozens of investment options1 the two main ones to look at are: bonds: these are fixed income, which means they\'re fairly safe, but the downsi

In [ ]:
Ranked_documents("Which is the best stock available currently?")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Ranked Documents :


[('Price for the latest transaction. If the stock is selling for  $898.7 means that the stock is currently trading for  $898.7, and it will be your ask price of stock if you purchase currently.',
  [0.7418820523308783]),
 ('If you are not worried about timing the market and want to buy primarily "blue chip" stocks to hold for a while, consider using Loyal3. They don\'t charge any commission. The downside is that trades are executed at the end of the day and there\'s only about 60 companies currently available (but there are some really good ones currently available).',
  [0.7325417822012723]),
 ("There are 2 primary ways to bet against a stock if you think it will decline. The first is to short sell shares of that stock the second is to buy put options (I would also add that selling naked call options would also be a bet against but I don't believe that is as common as the other 2 mentioned methods). The problem with short selling an IPO is that you first have to borrow the shares you 

In [ ]:
Ranked_documents("What are the risks of Dividend-yielding stocks?")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Ranked Documents :


[('You\'re talking about money in a savings account, and avoiding the risks posed by an ongoing crisis, and avoiding risk. If you are risk-averse, and likely to need your money in the short term, you should not put your money in the stock market, even in "safe" stocks like P&G/Coca-Cola/etc. Even these safe stocks are at risk of wild price swings in the short- to intermediate-term, especially in the event of international crises such as major European debt defaults and the like. These stocks are suitable for long-term growth objectives, but they are not as a replacement for a savings account. Coca-Cola lost a third of its value between 2007 and 2009. (It\'s recovered, and is currently doing better than ever.) P&G went from $74/share to $46/share. (It\'s partially recovered and back at $63). On the other hand, these stocks may indeed be suitable as long-term investments to protect you against local currency inflation. And yes, they even pay dividends. If you\'re after this investment, a

In [ ]:
Ranked_documents("In what age we should start investing?")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Ranked Documents :


[("First off, you have done very well to be in your financial position at your age. Congratulations.  I first started investing seriously about 10 years ago, and when I started, I had a similar attitude to you. Learning how to invest is a journey, and it will take you a while to learn both the intellectual and emotional sides of investing. First off, there is nothing wrong with having a chunk of cash that you aren't investing effectively. It is far better to be losing earning power WRT inflation that it is to make a bad investment, where you can lose all your money quite quickly. I have perhaps 15% of my capital just sitting around right now because I don't have any place where I'm excited to put it. For your IRA, I would look at the options you have, and choose one that is reasonably well diversified and has low costs. In most cases, an index fund is a reasonable choice. My 401K goes into an S&P 500 index fund, and I don't have to worry about it. Beyond that, I suggest spending some t